# Test of GraphSAGE
- the version2 of larger hidden dimension
- use DGL
- predict `graphs`
- valid, test data are in the training dataset

In [1]:
import os
import dgl
import json
import torch
import torch as th
import dgl.nn as dglnn
# from tqdm import tqdm
from tqdm.notebook import tqdm  # 使用 notebook 版本的 tqdm
import torch.nn as nn
from dgl.nn import GraphConv, GATConv, SAGEConv
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup

- check the GPU and assign the GPU by the best memory usage

In [2]:
import subprocess
import torch

def get_free_gpu():
    try:
        # Run nvidia-smi command to get GPU details
        _output_to_list = lambda x: x.decode('ascii').split('\n')[:-1]
        command = "nvidia-smi --query-gpu=memory.free --format=csv,nounits,noheader"
        memory_free_info = _output_to_list(subprocess.check_output(command.split())) 
        memory_free_values = [int(x) for i, x in enumerate(memory_free_info)]
        
        # Get the GPU with the maximum free memory
        best_gpu_id = memory_free_values.index(max(memory_free_values))
        return best_gpu_id
    except:
        # If any exception occurs, default to GPU 0 (this handles cases where nvidia-smi isn't installed)
        return 0

if torch.cuda.is_available():
    # Get the best GPU ID based on free memory and set it
    best_gpu_id = get_free_gpu()
    device = torch.device(f"cuda:{best_gpu_id}")
else:
    device = torch.device("cpu")
    print("there's no available GPU")

# device = torch.device(f"cuda:{1}")
print(device)


cuda:3


## Fix the seed

In [3]:
import numpy as np
import torch
import random

#fix seed
def same_seeds(seed = 8787):
    torch.manual_seed(seed)
    # random.seed(seed) 
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## Data Loader

In [4]:
class GraphDataset(Dataset):
    def __init__(self, data_list, device):
        self.data_list = data_list
        self.device = device

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data = self.data_list[idx]
        return data

def collate(samples):
    data_list = samples
    batched_graphs = []
    for data in data_list:
        g = dgl.graph((th.tensor(data["edge_index"][0]), th.tensor(data["edge_index"][1])), num_nodes=data["num_nodes"])

        g.ndata['feat'] = th.tensor(data["node_feat"])
        g.edata['feat'] = th.tensor(data["edge_attr"])
        g.edata['label'] = th.tensor(data["labels"])  # Add edge labels to graph

        batched_graphs.append(g)
    
    return dgl.batch(batched_graphs)

In [5]:
datasets = ['repeat_train_320', 'valid', 'test']
# datasets = ['valid']
dataset_data = {}

for dataset_name in tqdm(datasets):
    file_path = f"../../data_processing/dgl/data_new/exp3/training_data_repeat/transR_50/{dataset_name}.jsonl"
    
    print(file_path)
    with open(file_path) as f:
        data_list = [json.loads(line) for line in tqdm(f, position=0, leave=True)]
    
    
    dataset_data[dataset_name] = GraphDataset(data_list, device)

print("Datasets loaded!")

  0%|          | 0/3 [00:00<?, ?it/s]

../../data_processing/dgl/data_new/exp3/training_data_repeat/transR_50/repeat_train_320.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp3/training_data_repeat/transR_50/valid.jsonl


0it [00:00, ?it/s]

../../data_processing/dgl/data_new/exp3/training_data_repeat/transR_50/test.jsonl


0it [00:00, ?it/s]

Datasets loaded!


- choose batch size

In [6]:
def create_dataloaders(batch_size, shuffle=True):
    dataloaders = {}
    for dataset_name, dataset in dataset_data.items():
        # do not shuffle the testing dataset
        if dataset_name == "test":
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate)    
        else:
            dataloaders[dataset_name] = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate)
    return dataloaders

dataloaders = create_dataloaders(64)

- Turn the print message to a log file

In [8]:
import datetime

now = datetime.datetime.now()

formatted_time = now.strftime("%m%d_%H:%M")

log_file_path = f"../log_message/{formatted_time}_GraphSAGE_transR_50.log"

def add_log_msg(msg, log_file_path=log_file_path):
    with open(log_file_path, 'a') as f:
        f.write(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}\n')
    print(f'{datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}# {msg}')

print(log_file_path)

../log_message/1017_10:48_GraphSAGE_transR_50.log


### Model

In [9]:
class GraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GraphSAGE, self).__init__()
        self.layer1 = dglnn.SAGEConv(in_dim, hidden_dim, 'pool')
        self.layer2 = dglnn.SAGEConv(hidden_dim, out_dim, 'pool')
        self.dropout = nn.Dropout(0.25)

    def forward(self, g, inputs):
        h = self.layer1(g, inputs)
        h = torch.relu(h)
#         h = self.dropout(h)
        h = self.layer2(g, h)
        return h

In [10]:
class MLPPredictor(nn.Module):
    def __init__(self, out_feats, out_classes):
        super().__init__()
        self.W = nn.Linear(out_feats*2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        score = self.W(torch.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [11]:
class Model(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_classes):
        super().__init__()
        self.sage = GraphSAGE(in_features, hidden_features, out_features)
        self.pred = MLPPredictor(out_features, num_classes)
      
    def forward(self, g, node_feat, return_logits=False):
        h = self.sage(g, node_feat)
        logits = self.pred(g, h)
        
        return logits

- Model Forward  

In [18]:
def model_fn(batched_g, model, criterion, device, count=1, which_type='train'):
    """Forward a batch through the model."""
#     batched_g, labels = data
    batched_g = batched_g.to(device)
    
    labels = batched_g.edata['label'].to(device)
    
    logits = model(batched_g, batched_g.ndata['feat'].float())

    loss = criterion(logits, labels)

    output = torch.softmax(logits, dim=1)
    preds = output.argmax(1)
    
    # Compute accuracy
    accuracy = torch.mean((preds == labels).float())
    
    if which_type == 'validation' and count % 1000 == 0:
        add_log_msg(f"labels of Validation: {labels} {labels.shape}")
        add_log_msg(f"predicted of Validation: {preds} {preds.shape}")
        
    elif which_type == 'test'  and count % 1000 == 0:
        add_log_msg(f"labels of Test: {labels} {labels.shape}")
        add_log_msg(f"predicted of Test: {preds} {preds.shape}")
        
    if count % 50000 == 0: 
        add_log_msg(f"labels of {count}: {labels} {labels.shape}")
        add_log_msg(f"predicted of {count}: {preds} {preds.shape}")
        
    return loss, accuracy, preds

### Training

- Fix the seed and save the model.state_dict that contains the initial weight

In [13]:
seed = 8787
same_seeds(seed)

model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
torch.save(model.state_dict(), 'model3_initial(graphsage)/initial_weight.pth')

In [14]:
# model.layer1.fc_self.weight
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[-0.0682,  0.0153, -0.1769,  ...,  0.0375,  0.2321, -0.2812],
        [-0.2271,  0.2290, -0.1997,  ..., -0.0095,  0.1509,  0.2686],
        [-0.2743,  0.0406, -0.1222,  ...,  0.1036, -0.1590, -0.2555],
        ...,
        [-0.0758,  0.0461,  0.1273,  ...,  0.1367,  0.0671, -0.2605],
        [-0.2425, -0.1362,  0.2474,  ..., -0.3221, -0.0595,  0.3141],
        [ 0.0234, -0.2783,  0.2146,  ..., -0.3020, -0.1751,  0.0528]],
       requires_grad=True)

- Check if model really load the model_dict

In [15]:
model = Model(in_features=50, hidden_features=64, out_features=128, num_classes=167)
model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
model.sage.layer1.fc_self.weight

Parameter containing:
tensor([[-0.0682,  0.0153, -0.1769,  ...,  0.0375,  0.2321, -0.2812],
        [-0.2271,  0.2290, -0.1997,  ..., -0.0095,  0.1509,  0.2686],
        [-0.2743,  0.0406, -0.1222,  ...,  0.1036, -0.1590, -0.2555],
        ...,
        [-0.0758,  0.0461,  0.1273,  ...,  0.1367,  0.0671, -0.2605],
        [-0.2425, -0.1362,  0.2474,  ..., -0.3221, -0.0595,  0.3141],
        [ 0.0234, -0.2783,  0.2146,  ..., -0.3020, -0.1751,  0.0528]],
       requires_grad=True)

In [16]:
mapping_file = './new_mapping.txt'
label_mapping = {}
with open(mapping_file, 'r') as f:
    for line in f:
        parts = line.strip().split(': ')
        label_mapping[int(parts[1])] = parts[0]

In [17]:
import csv
import pandas as pd
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 5269
same_seeds(seed)

model = Model(in_features=50, hidden_features=256, out_features=512, num_classes=167)
# model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
best_model_path = "../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50_hidden_dim_2.pt"

model = model.to(device)

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
optimizer = AdamW(model.parameters(), lr=5e-4)
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=18, num_training_steps=total_steps)

# T_max control the period of the lr changing -> set 1/10 first
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)

criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.BCEWithLogitsLoss()

total_steps = 7

# save the best model
best_val_loss = float('inf')
patience = 4  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.
early_stop = False

# Training Part
for epoch in tqdm(range(total_steps)):
    # Train
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0

    for batched_g in tqdm(dataloaders['repeat_train_320'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

        
#     scheduler.step()
    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')

    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0


    with torch.no_grad():
        for batched_g in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)

    else:
        waiting += 1
        if waiting >= patience:
            early_stop = True
            
    if epoch == total_steps - 1 or early_stop == True:  # Last epoch
        pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50_hidden_dim_2.pt'
        model.load_state_dict(torch.load(pretrained_model_path))
        model.to(device)
        
        true_labels = []
        predicted_labels = []

        with torch.no_grad():
            for batched_g in tqdm(dataloaders['repeat_train_320'], desc="Generating Train Report", position=0, leave=True):
                loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, which_type='train')
                labels = batched_g.edata['label'].to(device)
                
                true_labels.extend(labels.cpu().numpy())
                predicted_labels.extend(predicted.cpu().numpy())

        # Assuming that the mapping and other things are defined earlier or globally
        mapped_true_labels = [label_mapping[label] for label in true_labels]
        mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

        # Generate classification report
        train_report = classification_report(mapped_true_labels, mapped_predicted_labels)
        add_log_msg(f"Train Classification Report at Epoch {epoch}:\n{train_report}")

    
    if early_stop:
        add_log_msg("============================== Early stopping ==================================")
        break 

  0%|          | 0/7 [00:00<?, ?it/s]

Training:   0%|          | 0/213400 [00:00<?, ?it/s]

10/17/2023, 10:50:46# labels of 5000: tensor([ 38,   9,  24,  75, 164,  83,  24,  24, 163, 125, 125,  12,  31,  74,
        150,   2, 150,  33,  48,  11, 158,  36,  18, 116, 121,   2,   4, 151,
        142, 121,  47,  48,  55,   4,  34,  30,  11,  48, 144, 164, 125, 157,
         34,  87,  55,   9,  48,  31,  12, 125, 150,  47,  48,  48,  49, 151,
        158,  55, 163,  74,  34,  14,  81, 142], device='cuda:3') torch.Size([64])
10/17/2023, 10:50:46# predicted of 5000: tensor([125,  30,  30, 125, 125,  30, 125, 125, 125,  30,   9,  30, 125,  30,
        125,  30, 125,  30,   9,   9,  30,  30,  30, 125, 125,   9, 125, 125,
         30, 125,  30, 125, 125,  30, 125,  30,  30,  30, 125,  30, 125, 125,
         30,  30, 125, 125,  30,  30, 125,   9, 125,  30, 125, 125, 125, 125,
         30, 125,  30, 125, 125,  30, 125,   9], device='cuda:3') torch.Size([64])
10/17/2023, 10:53:25# labels of 10000: tensor([  2,  53,  24,  38, 162, 163, 121,  87,   1, 164,  38,  42,  31, 144,
        157,  

10/17/2023, 11:19:16# labels of 55000: tensor([ 54,  47,  31,  42, 151,  60,   1,  14, 112, 163,   9, 119, 144,  74,
         33,  42,  53, 104,  44,   2, 152,  49, 109,  24,  18,  57, 151, 152,
        116, 142, 143, 150, 151,  53,  87,  36, 163,   4, 111,  57,  53, 150,
         48, 125,  55,  53,  44, 143, 143,  38,  60,  92,  57, 143,  14,  44,
         60,  33,  42,  33,  76,   1, 111,  14], device='cuda:3') torch.Size([64])
10/17/2023, 11:19:16# predicted of 55000: tensor([ 54,  87, 152,  12,  42,  33,  34,  97,  54,  11,   9,   1, 152,  74,
         30,  55,  53, 104,  44, 116,  12,   1, 119,  33, 121,  74, 151, 121,
         53,  53,  74,  34, 163, 104, 144,  36, 163, 119,  76,  24,  33,  34,
         60, 158, 124, 152,  24, 152,  30,  38, 158,  92, 112, 143,  57,  24,
         53, 116,  42,  12, 158, 142,  44,  48], device='cuda:3') torch.Size([64])
10/17/2023, 11:22:01# labels of 60000: tensor([152,  33,  87, 119, 150, 163,  57,  60, 150, 119,  74, 143, 112,  76,
         30,

10/17/2023, 11:43:36# labels of 100000: tensor([111, 164,  36, 164,  83,  83,  14,  75,  97,  53,  76,  14,  92,  81,
          4,  76,  18, 144, 152, 164,  11, 125,  48, 152, 121,   1,  34,  54,
         49,  83, 164,  53, 125,  30,  44,  74, 158,  11,  92, 124,  42, 158,
        109, 111,  11,  48,   9, 104,  33, 158,  33,  81, 110, 110,  38, 152,
         49, 162, 121,  48,  34,   9,  14,  44, 119], device='cuda:3') torch.Size([65])
10/17/2023, 11:43:36# predicted of 100000: tensor([ 47, 158,  36, 164,  74, 112, 121,  75, 125,  11, 158,  14,  92, 150,
          4,  54,  47, 119, 152,  60, 125, 142,  48, 116, 121, 112, 109,  54,
         74, 158,  42,  11, 104,  49,   1,  87, 151,  47,  92, 124, 111, 158,
        109, 112,  11, 104, 109, 119,  60,  18, 151, 157,  53,  53,  54,  11,
         49, 158, 121,  42,  31,  48,  14,  31, 109], device='cuda:3') torch.Size([65])
10/17/2023, 11:46:18# labels of 105000: tensor([152,  42, 163,   9,  47,  44, 150, 164,  42, 163, 125,  42,   2,  54,

...

Training:   0%|          | 0/213400 [00:00<?, ?it/s]

10/17/2023, 21:00:37# labels of 5000: tensor([ 34,   9, 109, 111,  44, 162,  53, 112,  87,  24,  36, 109, 104,  14,
        150, 162,  75, 158, 109,  24,  44,  12,  42, 157, 121,  92,  81,  54,
         33,  34, 151,  83,  12, 158, 144, 151,  47,  18, 104,  76,  24, 111,
         49,  60,  75,  36,  47,  34,  75, 151,  36,  57,  83,  49, 143,  24,
        121, 104, 151,  24, 109,  34, 121, 152], device='cuda:3') torch.Size([64])
10/17/2023, 21:00:37# predicted of 5000: tensor([ 34,  38, 109, 111,  44, 162,  53, 112, 164,  24,  53, 109, 111,  14,
         75, 162,  74,  42, 109,  24,  92,  12,  42, 157,  54,  92, 124, 157,
         33,  34,  60,  83,  12,  36,  92,  54,  31,  38,  48, 125,  24, 111,
         60, 112, 109,  36,  47,  34,  74, 151,  36,  57,  49,  49,  34,  24,
         38, 104, 151,  24, 109,  34, 121,  75], device='cuda:3') torch.Size([64])
10/17/2023, 21:03:28# labels of 10000: tensor([ 74,  30,  92,  24,  76,  44,  57, 151,  31, 144,  55,  76,  83,  53,
        121, 1

10/17/2023, 21:28:41# labels of 50000: tensor([  9, 142, 104,  11,  57, 109, 104, 151, 112,  92,   2,   2,  55, 116,
         49,  44,  14,  57,  83,  42, 158, 125, 151,   9,  12,  11,  11, 109,
         14,  48, 124,  97, 109, 119,  33,  38, 109,  44, 151,   4,  76,  11,
         75, 112, 119,  11,  18,  55, 143,  55, 142,  75,  18, 163, 163, 157,
         30,   1,   4, 116,   2, 152, 119, 121], device='cuda:3') torch.Size([64])
10/17/2023, 21:28:41# predicted of 50000: tensor([  9, 142, 104,  11,  57, 109, 104, 125, 112,  92,   2,   2,  55, 116,
         49,  48,  14,  11,  83,  42, 158, 125, 151,   9,  57,  87,  47, 109,
         14,  48, 164,  97, 109, 119, 142,  38, 109,  92, 151, 152,  76, 158,
         55, 112, 119,  11, 162,  55, 143,  55, 142,  92, 162, 163, 163, 157,
         30,  31,   4, 116,   2, 152,  53, 121], device='cuda:3') torch.Size([64])
10/17/2023, 21:31:35# labels of 55000: tensor([ 75,  11,  92, 125, 124, 144,  75,  33,   4, 119, 112, 151,  18,  76,
        119,

10/17/2023, 21:57:18# labels of 100000: tensor([  4, 157,  55, 157,  57,  11,  48,  72,  72, 151,  55,  30,  42,  34,
         81,  48,  87,  57,   2,   9, 119, 124, 152,   9, 150,  92, 143,  49,
         33,  49,  14,  55, 119, 142, 119,  97, 104, 151,   2, 116,  31, 119,
         87,  57,  83,  11,  49,  60,  31,  87,   1,  24,  92, 124,  14,  92,
         44, 157,  14, 116,  36,  76,  34, 121, 104], device='cuda:3') torch.Size([65])
10/17/2023, 21:57:18# predicted of 100000: tensor([  4, 157, 125, 157,  57,  11,  42,  11,   2, 151,  55,  30,  42,  34,
         81, 142, 143,  57, 109,   9, 119, 124, 152,   9, 150,  92, 143,   4,
         48, 150,  14,  55, 119, 142, 119,  97, 104, 151,   2,  60,  31, 119,
         87,  14, 157,  11,  60,  60,  31,  38,   1,  24,  92, 124,   9,  92,
         44, 157,  44, 116,  36, 121,  34,   9, 104], device='cuda:3') torch.Size([65])
10/17/2023, 22:00:10# labels of 105000: tensor([ 60,  24,   9, 151, 104,  81, 119,  38,   1,  53, 112,  18,   1,  48,

KeyboardInterrupt: 

In [20]:
import csv
import pandas as pd
from sklearn.metrics import classification_report
from torch.optim import AdamW, lr_scheduler

seed = 5269
same_seeds(seed)

# model = Model(in_features=50, hidden_features=256, out_features=512, num_classes=167)
# model.load_state_dict(torch.load('model3_initial(graphsage)/initial_weight.pth'))
best_model_path = "../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50_hidden_dim.pt"
model.load_state_dict(torch.load(best_model_path))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-4)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=36, eta_min=0, last_epoch=- 1, verbose=False)

total_steps = 7
best_val_loss = float('inf')
patience = 3  # Number of epochs with no improvement after which training will be stopped.
waiting = 0  # The number of epochs with no improvement so far.
early_stop = False

# Training Part
for epoch in tqdm(range(total_steps)):
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0

    for batched_g in tqdm(dataloaders['repeat_train_320'], desc="Training", position=0, leave=True):
        num_batches += 1
        loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='train')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_accuracy += accuracy.item()

    add_log_msg(f"total batches: {num_batches}")

    avg_loss = total_loss / num_batches
    avg_accuracy = total_accuracy / num_batches

    add_log_msg(f'Epoch {epoch} | Train Loss: {avg_loss:.4f} | Train Accuracy: {avg_accuracy:.4f}')
    
    # Validation Part
    model.eval()
    total_accuracy = 0.0
    total_loss = 0.0
    num_batches = 0

    with torch.no_grad():
        for batched_g in tqdm(dataloaders['valid'], desc="Validation", position=0, leave=True):
            loss, accuracy, _ = model_fn(batched_g, model, criterion, device, num_batches, which_type='validation')
            total_accuracy += accuracy.item()
            total_loss += loss.item()
            num_batches += 1

    avg_accuracy = total_accuracy / num_batches
    current_loss = total_loss / num_batches
    
    add_log_msg(f'Validation Loss: {current_loss:.4f} | Validation Accuracy: {avg_accuracy:.4f}\n')
    
    if current_loss < best_val_loss:
        best_val_loss = current_loss
        waiting = 0
        
        if os.path.exists(best_model_path):
            os.remove(best_model_path)
            add_log_msg("Find a better model!!")

        torch.save(model.state_dict(), best_model_path)

    else:
        waiting += 1
        if waiting >= patience:
            early_stop = True
            
    if epoch == total_steps - 1 or early_stop == True:  # Last epoch
        model.load_state_dict(torch.load(best_model_path))
        model.to(device)
        
        true_labels = []
        predicted_labels = []

        with torch.no_grad():
            for batched_g in tqdm(dataloaders['repeat_train_320'], desc="Generating Train Report", position=0, leave=True):
                loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, which_type='train')
                labels = batched_g.edata['label'].to(device)
                
                true_labels.extend(labels.cpu().numpy())
                predicted_labels.extend(predicted.cpu().numpy())

        # Assuming that the mapping and other things are defined earlier or globally
        mapped_true_labels = [label_mapping[label] for label in true_labels]
        mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

        # Generate classification report
        train_report = classification_report(mapped_true_labels, mapped_predicted_labels)
        add_log_msg(f"Train Classification Report at Epoch {epoch}:\n{train_report}")
    
    
    if early_stop:
        add_log_msg("============================== Early stopping ==================================")
        break 

  0%|          | 0/7 [00:00<?, ?it/s]

Training:   0%|          | 0/213400 [00:00<?, ?it/s]

10/17/2023, 22:43:17# labels of 50000: tensor([ 76,  33,   1,  44,  92, 124,  34,   9, 124,  55,  97,   4,  74,   1,
        152, 142,  83, 119, 143,  55, 104,  47, 143,  60, 121,  31, 109,  74,
        109, 152,  87,  31, 124,  97,  44,  42, 125, 144,  81,  31, 119,  83,
        157,  31, 163,   1,  42, 124,  97,  76,  30,  48,  53, 157, 143, 158,
        119, 164,   4,  83,  48,   2,  42, 116], device='cuda:3') torch.Size([64])
10/17/2023, 22:43:17# predicted of 50000: tensor([  1,  33,   1, 152,  48,  53, 112,   9, 124,  55,  97,   4,  74,   1,
        152,  83,  83, 119, 143,  55, 116,  47, 143,  81, 121,  31, 109,  74,
        116, 152, 119,  31, 144, 150, 162,  34, 125, 144,  81,  31, 119,  83,
        157,  31, 112,   1,  42, 124,  47,  55,  81, 116,  31, 157, 150, 158,
        119, 164,  33,  76,   4,   2,  42, 116], device='cuda:3') torch.Size([64])
10/17/2023, 23:13:13# labels of 100000: tensor([ 48,  36,  74, 125, 157, 150,  42,  53,  55,  55,  34,  38, 157,  49,
         92

Validation:   0%|          | 0/258 [00:00<?, ?it/s]

10/18/2023, 00:19:06# labels of Validation: tensor([20, 20, 72,  ..., 69, 69, 69], device='cuda:3') torch.Size([1970])
10/18/2023, 00:19:06# predicted of Validation: tensor([20, 20, 53,  ..., 69, 69, 69], device='cuda:3') torch.Size([1970])
10/18/2023, 00:19:06# labels of 0: tensor([20, 20, 72,  ..., 69, 69, 69], device='cuda:3') torch.Size([1970])
10/18/2023, 00:19:06# predicted of 0: tensor([20, 20, 53,  ..., 69, 69, 69], device='cuda:3') torch.Size([1970])
10/18/2023, 00:19:16# Validation Loss: 0.3735 | Validation Accuracy: 0.9557

10/18/2023, 00:19:16# Find a better model!!


Training:   0%|          | 0/213400 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
model.load_state_dict(torch.load(best_model_path))
model.to(device)

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['repeat_train_320'], desc="Generating Train Report", position=0, leave=True):
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, which_type='train')
        labels = batched_g.edata['label'].to(device)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

# Assuming that the mapping and other things are defined earlier or globally
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# Generate classification report
train_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"Train Classification Report at Epoch {epoch}:\n{train_report}")

Generating Train Report:   0%|          | 0/213400 [00:00<?, ?it/s]

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


10/18/2023, 02:34:39# Train Classification Report at Epoch 1:
                                                precision    recall  f1-score   support

T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554       1.00      1.00      1.00     51200
    T1003.001_35d92515122effdd73801c6ac3021da7       1.00      1.00      1.00      4800
    T1003.002_5a484b65c247675e3b7ada4ba648d376       1.00      1.00      1.00      4000
    T1003.002_7fa4ea18694f2552547b65e23952cabb       1.00      1.00      1.00     12000
    T1003.003_9f73269695e54311dd61dc68940fb3e1       0.63      0.82      0.71    256000
    T1003.003_f049b89533298c2d6cd37a940248b219       0.72      0.58      0.64    256000
        T1003_18f31c311ac208802e88ab8d5af8603e       1.00      1.00      1.00      4800
        T1007_9d03c91bdae5a80f17f89c987942b5a8       1.00      1.00      1.00      4800
    T1007_c6607391-d02c-44b5-9b13-d3492ca58599       0.68      0.61      0.64    256000
        T1007_d6bb2a19da7246731ed9c44831b135f8       0.00

In [25]:
# train_report

In [23]:
train_report = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(train_report).transpose()

report_df

/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/workdir/home/euni/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554,1.000000,1.000000,1.000000,5.120000e+04
T1003.001_35d92515122effdd73801c6ac3021da7,1.000000,1.000000,1.000000,4.800000e+03
T1003.002_5a484b65c247675e3b7ada4ba648d376,1.000000,1.000000,1.000000,4.000000e+03
T1003.002_7fa4ea18694f2552547b65e23952cabb,1.000000,1.000000,1.000000,1.200000e+04
T1003.003_9f73269695e54311dd61dc68940fb3e1,0.625686,0.823750,0.711185,2.560000e+05
...,...,...,...,...
T1574.011_72249c1e9ffe7d8f30243d838e0791ca,1.000000,0.999792,0.999896,4.800000e+03
benign,0.999994,0.999998,0.999996,1.077033e+06
accuracy,0.697473,0.697473,0.697473,6.974732e-01
macro avg,0.804047,0.786129,0.786919,1.600823e+07


In [41]:
report_df.to_csv('hidden_dim_2.csv', index=True)

report_df

,precision,recall,f1-score,support
T1003.001_0ef4cc7b-611c-4237-b20b-db36b6906554,1.000000,1.000000,1.000000,5.120000e+04
T1003.001_35d92515122effdd73801c6ac3021da7,1.000000,1.000000,1.000000,4.800000e+03
T1003.002_5a484b65c247675e3b7ada4ba648d376,1.000000,1.000000,1.000000,4.000000e+03
T1003.002_7fa4ea18694f2552547b65e23952cabb,1.000000,1.000000,1.000000,1.200000e+04
T1003.003_9f73269695e54311dd61dc68940fb3e1,0.625686,0.823750,0.711185,2.560000e+05
...,...,...,...,...
T1574.011_72249c1e9ffe7d8f30243d838e0791ca,1.000000,0.999792,0.999896,4.800000e+03
benign,0.999994,0.999998,0.999996,1.077033e+06
accuracy,0.697473,0.697473,0.697473,6.974732e-01
macro avg,0.804047,0.786129,0.786919,1.600823e+07


In [43]:
filtered_report = report_df[(report_df['support'] == 256000)] # 256000 = 100 x 8 x 320 -> single triplet ones
num_rows = filtered_report.shape[0]
print("Number of single triplet:", num_rows)

filtered_report = report_df[(report_df['f1-score'] <= 0.2) & (report_df['support'] == 256000)]
num_rows = filtered_report.shape[0]
print("Number of single triplet and f1-score<=0.2:", num_rows)

filtered_report = report_df[(report_df['f1-score'] <= 0.5) & (report_df['support'] == 256000)]
num_rows = filtered_report.shape[0]
print("Number of single triplet and f1-score<=0.5:", num_rows)

filtered_report = report_df[(report_df['f1-score'] > 0.5) & (report_df['support'] == 256000)]
num_rows = filtered_report.shape[0]
print("Number of single triplet and f1-score>0.5:", num_rows)

Number of single triplet: 53
Number of single triplet and f1-score<=0.2: 0
Number of single triplet and f1-score<=0.5: 1
Number of single triplet and f1-score>0.5: 52


In [32]:
import matplotlib.pyplot as plt

filtered_report = report_df[(report_df['support'] == 256000)]
# filtered_report[['precision', 'recall', 'f1-score']].plot(kind='bar', figsize = (12,8))
filtered_report[['f1-score']].plot(kind='bar', figsize = (16,8))
plt.show()

In [40]:
ax = filtered_report[['f1-score']].plot(kind='bar', figsize=(22,10))

# 添加 f1-score 值到 bar 上方
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2f'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', 
                va = 'center', 
                xytext = (0, 9), 
                textcoords = 'offset points')

plt.show()

### test of valid and test part is ``graph``

- 60 APs in training x 10000times
- 5 APs in validation x 4 times
- 3 APs in test x 4 times
- Batch size = 4

In [ ]:
# load the pretrained model
pretrained_model_path = '../checkpoint_graphSAGE/best_model_GraphSAGE_transR_50_hidden_dim.pt'
model.load_state_dict(torch.load(pretrained_model_path))

model.to(device)
model.eval()

total = 0
correct = 0
count = 0

true_labels = []
predicted_labels = []

with torch.no_grad():
    for batched_g in tqdm(dataloaders['test'], desc="Testing", position=0, leave=True):
#         print(f"data:{data[1]}")
        loss, accuracy, predicted = model_fn(batched_g, model, criterion, device, count, which_type='test')
        labels = batched_g.edata['label'].to(device)
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())
        
        if count % 5000 == 0:
            add_log_msg(f"labels: {labels} {labels.shape}")
            add_log_msg(f"predicted: {predicted} {predicted.shape}")
            
        count += 1
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

add_log_msg(f'Test Accuracy: {100 * correct / total} %\n\n\n')


# ======================================== handlig the output excel files ========================================
# mapping_file = './new_mapping.txt'
# label_mapping = {}
# with open(mapping_file, 'r') as f:
#     for line in f:
#         parts = line.strip().split(': ')
#         label_mapping[int(parts[1])] = parts[0]
        
# 将映射后的标签应用到true和predicted标签列表
mapped_true_labels = [label_mapping[label] for label in true_labels]
mapped_predicted_labels = [label_mapping[label] for label in predicted_labels]

# 生成Scikit-learn报告信息的DataFrame
report_data = classification_report(mapped_true_labels, mapped_predicted_labels, output_dict=True)
report_df = pd.DataFrame(report_data).transpose()

# mapped_true_labels_np = np.array(mapped_true_labels)
# mapped_predicted_labels_np = np.array(mapped_predicted_labels)

# print("mapped_true_labels 的形状:", mapped_true_labels_np.shape)
# print("mapped_predicted_labels 的形状:", mapped_predicted_labels_np.shape)

report_folder = 'classification_report'
os.makedirs(report_folder, exist_ok=True)

count = 0
while True:
    report_filename = f'classification_report-transE_50-graphSAGE-{count}.xlsx'
    labels_filename = f'mapped_true_predicted_labels-transE_50-graphSAGE-{count}.xlsx'
    
    report_path = os.path.join(report_folder, report_filename)
    labels_path = os.path.join(report_folder, labels_filename)
    
    if not os.path.exists(report_path) and not os.path.exists(labels_path):
        break
    count += 1

    
report_df.to_excel(report_path, index_label='Label')

mapped_labels_df = pd.DataFrame({'true_label': mapped_true_labels, 'predicted_label': mapped_predicted_labels})
mapped_labels_df.to_excel(labels_path, index=False)

add_log_msg(f"report path: {report_path}")
add_log_msg(f"label path: {labels_path}")

mapped_report = classification_report(mapped_true_labels, mapped_predicted_labels)
add_log_msg(f"mapped_report:\n{mapped_report}")